In [6]:
import numpy as np
from math import factorial
from cmath import exp


def simplex_content(j, n_dims, signed, *args):
    """
    Compute simplex content (j-dim-volume) using Cayley-Menger Determinant
    :param j: dimension of simplex
    :param n_dims: dimension of R^n space
    :param signed: bool denoting whether to calculate signed content or unsigned content
    :param args: v0, v1, ... vectors for the coordinates of the vertices defining the simplex

    :return: vol: volume of the simplex
    """
    assert(n_dims == len(args[0]))
    assert(isinstance(signed, bool))
    n_vert = len(args)
    assert(n_vert == j+1)

    if n_dims > j:
        assert(not signed)

    if not signed:
        # construct Cayley-Menger matrix
        B = np.zeros([j+2, j+2])
        B[:, 0] = 1
        B[0, :] = 1
        B[0, 0] = 0
        for r in range(1, j+2):
            for c in range(r+1, j+2):
                vr = args[r-1]
                vc = args[c-1]
                B[r, c] = np.linalg.norm(vr-vc) ** 2
                B[c, r] = B[r, c]
        vol2 = (-1)**(j+1) / (2**j) / (factorial(j)**2) * np.linalg.det(B)
        if vol2 < 0:
            print("Warning: zeroing small negative number {0}".format(vol2))
        vol = np.sqrt(max(0, vol2))
    else:
        # matrix determinant
        mat = np.zeros([j, j])
        for r in range(j):
            mat[:, r] = args[r] - args[-1]
        vol = np.linalg.det(mat) / factorial(j)

    return vol


In [57]:
def simplex_ft_device(p, uvw):
    j = p.shape[0]-1
    n_dims = len(uvw)
    Fi = complex(0, 0)
    for dim in range(j+1):
        xi = p[dim]
        denom = 1
        uvw_dot_xi = complex(0, 0)
        for dd in range(n_dims):
            uvw_dot_xi += uvw[dd] * xi[dd]
        for d in range(j+1):
            if d != dim:
                xj = p[d]
                uvw_dot_xj = complex(0, 0)
                for dd in range(n_dims):
                    uvw_dot_xj += uvw[dd] * xj[dd]
                denom *= uvw_dot_xi - uvw_dot_xj
        Fi += exp(-1j*(uvw_dot_xi)) / denom
    return Fi

def pt_ft(V, E, k, j=2):
    k = k.copy() * 2 * np.pi
    F = complex(0,0)
    j = 2
    for ie in range(E.shape[0]):
        gamma = factorial(2) * simplex_content(2, 2, False, V[E[ie, 0]], V[E[ie, 1]], V[E[ie, 2]])
        fac = complex(0,0)
        if not k[0] == k[1] == 0:
            for dim in range(j+1):
                other_dims = [d for d in range(j+1) if d != dim]
                denom = 1
                sig_i = V[E[ie, dim]].dot(k)
                for d in other_dims:
                    sig_j = V[E[ie, d]].dot(k)
                    denom *= sig_i - sig_j
                fac += np.exp(-1j*sig_i) / denom
        else:
            fac = -0.5
        F += fac * gamma * (-1)
        
    return F

def diff(V, E, k, pt, delta):
    V_p = V.copy(); V_p[pt] += delta
    V_m = V.copy(); V_m[pt] -= delta
    F_p = pt_ft(V_p, E, k)
    F_m = pt_ft(V_m, E, k)
    dFdV = (F_p - F_m) / 2 / delta
    return dFdV

def pt_ft_bw(V, E, k, pt):
    k = k.copy() * 2 * np.pi
    dFdV = np.zeros(2, dtype=np.complex_)
    j = 2
    # only loop over elems containing the point
    for ie in range(E.shape[0]):
        if pt[0] in E[ie]:
            sid = np.argwhere(E[ie]==pt[0])[0][0]
            seq = np.array([sid % 3, (sid+1) % 3, (sid+2) % 3])
            if not k[0] == k[1] == 0:
                gamma = factorial(2) * simplex_content(2, 2, True, V[E[ie, 0]], V[E[ie, 1]], V[E[ie, 2]])
                sig0 = V[E[ie, seq[0]]].dot(k)
                sig1 = V[E[ie, seq[1]]].dot(k)
                sig2 = V[E[ie, seq[2]]].dot(k)
                esig0 = np.exp(-1j*sig0)
                esig1 = np.exp(-1j*sig1)
                esig2 = np.exp(-1j*sig2)
                s01 = 1/(sig0 - sig1)
                s12 = 1/(sig1 - sig2)
                s20 = 1/(sig2 - sig0)
                S = -(esig0*s01*s20+esig1*s01*s12+esig2*s12*s20)
                fac = -esig0*s01*s20*(-1j-s01+s20)+esig1*s12*s01**2-esig2*s12*s20**2
                fac *= gamma
            else:
                fac = 0
                S = -0.5
            pt1 = V[E[ie, seq[1]]]
            pt2 = V[E[ie, seq[2]]]
            vec23 = np.array([pt1[1]-pt2[1], -pt1[0]+pt2[0]])
            dFdV += - fac * k - S * vec23
                
    return dFdV[pt[1]]

from transform import simplex_ft_gpu
def simplex_fd(V, E, k, pt, delta):
    assert(np.absolute(k[0])<128 and np.absolute(k[1])<128 and k[1]>=0)
    D = np.ones((E.shape[0], 1))
    V_p = V.copy()
    V_m = V.copy()
    V_p[pt[0], pt[1]] += delta
    V_m[pt[0], pt[1]] -= delta
    Freq_p = simplex_ft_gpu(V_p, E, D, (256,256),(1,1),2,mode='density')
    Freq_m = simplex_ft_gpu(V_m, E, D, (256,256),(1,1),2,mode='density')
    dV_fd = (Freq_p - Freq_m) / delta / 2
    if k[0] >= 0:
        i = int(k[0])
    else:
        i = int(256 + k[0])
    j = int(k[1])
    return dV_fd[i,j,0]

In [58]:
from time import time
import numpy as np

# V = np.array([[0.2,0.2],
#               [0.8,0.5],
#               [0.8,0.8],
#               [0.6,0.8]])
# V = V+1e-10*np.random.rand(*V.shape)
# E = np.array([[0,1,2],
#               [2,3,0]])
# E = np.array([[0,1,3],
#               [2,3,1]])
E = np.array([[0,1,4],
              [1,2,4],
              [2,3,4],
              [3,0,4]])
V = np.array([[0.39,0.23],
              [0.91312,0.55],
              [0.93,0.81],
              [0.61,0.69],
              [0, 0]])
V = V+1e-10*np.random.rand(*V.shape)
D = np.ones((E.shape[0], 1))
# E = np.array([[0,1],
#               [1,2],
#               [2,3],
#               [3,0]])

In [69]:
k = np.array([-34.,101.])
pt = (4,0)
print(diff(V, E, k, pt, 1e-6))
print(diff(V, E, k, pt, 1e-7))
print(diff(V, E, k, pt, 1e-9))
print(simplex_fd(V, E, k, pt, 1e-6)/(256**2))
print(pt_ft_bw(V, E, k, pt))
# pt = (2,1)
# print(pt_ft_bw(V, E, k, pt))
# k = np.array([8.,0.])
# print(diff(V, E, k, (0,1), 1e-6))
# print(diff(V, E, k, (0,1), 1e-7))
# print(diff(V, E, k, (0,1), 1e-8))
# print(pt_ft_bw(V, E, k, (0,1)))

(1.3621899256092886e-06-0.002873712488498238j)
(1.3621993175106078e-06-0.002873712507986772j)
(1.3630250899308858e-06-0.0028737122612630147j)
(1.3621899289974204e-06-0.0028737124884914617j)
(4.0234064994579266e-21-4.336808689942018e-19j)


In [ ]:
P = V[E[1]]
x1, x2, x3 = P[0,0], P[1,0], P[2,0]
y1, y2, y3 = P[0,1], P[1,1], P[2,1]

In [ ]:
x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2)

In [ ]:
np.argwhere(np.array([1,8,4])==8)[0][0]